In [45]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [46]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [47]:
tf.keras.backend.clear_session()

In [48]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [49]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [50]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Amapá - Produção de Cimento (t)', axis=1)
data

,Ano,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade,Amapá - Consumo de Cimento (t)
0,2008,0.685967,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.281094,0.717218,5.483335e+08,2.546644e+09,...,0.679138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.791218,116.3660
1,2009,0.687230,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.302950,0.717750,5.302764e+08,2.679945e+09,...,0.678150,23.388804,1009.883647,1230.481450,25.905555,23.095009,85.784466,0.981601,0.793078,112.7290
2,2010,0.688267,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.270873,0.718079,5.046425e+08,2.891930e+09,...,0.677213,20.607778,1009.759833,1208.062836,25.916677,23.199352,86.045978,0.939026,0.794442,151.5420
3,2011,0.689067,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.258737,0.718205,4.854622e+08,3.091988e+09,...,0.676323,19.725663,1009.881525,1245.286694,25.844771,23.091132,85.930050,0.977264,0.795319,161.9700
4,2012,0.688823,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,14.118134,0.717668,4.867661e+08,3.189506e+09,...,0.675105,19.226938,1010.130132,1288.178519,25.905589,23.183123,86.045636,1.031823,0.795637,261.1200
5,2013,0.691309,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,14.922159,0.718558,5.225848e+08,3.093872e+09,...,0.675280,17.566310,1010.375134,1304.077775,25.869470,23.293712,86.546239,1.045650,0.795607,250.5190
6,2014,0.689418,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,15.577783,0.717271,5.676413e+08,2.779340e+09,...,0.673970,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.795157,227.2430
7,2015,0.689536,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,16.255907,0.716627,5.179392e+08,2.561731e+09,...,0.672885,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.794254,198.0696
8,2016,0.693917,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,16.439693,0.719375,4.524144e+08,2.516859e+09,...,0.668385,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.803514,166.8504
9,2017,0.692546,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,16.090418,0.718136,4.213689e+08,2.497710e+09,...,0.667148,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.802470,135.0210


In [51]:
input_data = data.iloc[:-2, 1:-1]
input_data = input_data.fillna(0.0)
input_data

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,0.685967,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.281094,0.717218,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.679138,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.791218
1,0.687230,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.302950,0.717750,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.678150,23.388804,1009.883647,1230.481450,25.905555,23.095009,85.784466,0.981601,0.793078
2,0.688267,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.270873,0.718079,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.677213,20.607778,1009.759833,1208.062836,25.916677,23.199352,86.045978,0.939026,0.794442
3,0.689067,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.258737,0.718205,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.676323,19.725663,1009.881525,1245.286694,25.844771,23.091132,85.930050,0.977264,0.795319
4,0.688823,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,14.118134,0.717668,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.675105,19.226938,1010.130132,1288.178519,25.905589,23.183123,86.045636,1.031823,0.795637
5,0.691309,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,14.922159,0.718558,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.675280,17.566310,1010.375134,1304.077775,25.869470,23.293712,86.546239,1.045650,0.795607
6,0.689418,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,15.577783,0.717271,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.673970,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.795157
7,0.689536,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,16.255907,0.716627,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.672885,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.794254
8,0.693917,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,16.439693,0.719375,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.668385,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.803514
9,0.692546,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,16.090418,0.718136,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.667148,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.802470


In [52]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     112.7290
1     151.5420
2     161.9700
3     261.1200
4     250.5190
5     227.2430
6     198.0696
7     166.8504
8     135.0210
9     123.3470
10    120.1530
11    133.5230
12    157.6210
13    142.7390
14         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [53]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,-1.598841,-1.819408,1.003478,1.202107,-1.961368,-1.393611,-0.283888,1.204752,-0.990234,0.166091,...,0.984445,1.322153,-2.822592,-3.316623,-3.299246,-3.314834,-3.316378,-3.309725,-3.276662,-1.489343
1,-1.043588,-1.675567,1.209427,1.179772,-1.951541,-1.387360,0.189618,0.820657,-0.546688,0.428911,...,-0.051710,1.112223,0.243196,0.299994,0.227232,0.318765,0.279390,0.248910,0.309186,-0.983518
2,-0.587693,-1.094542,1.324177,1.230212,-0.558611,-1.396535,0.481754,0.275392,0.158672,0.847995,...,-0.115145,0.913093,-0.121339,0.299551,0.162982,0.320325,0.295635,0.259758,0.153655,-0.612738
3,-0.236388,-0.575726,1.019683,1.107266,0.259840,-1.400006,0.594474,-0.132596,0.824346,1.230291,...,0.633403,0.724018,-0.236966,0.299987,0.269663,0.310239,0.278786,0.254949,0.293343,-0.374174
4,-0.343429,-0.115211,0.708407,0.661426,0.781023,0.275967,0.116875,-0.104860,1.148829,1.382743,...,-0.646476,0.465267,-0.302338,0.300877,0.392588,0.318770,0.293109,0.259744,0.492650,-0.287787
5,0.748761,0.286253,0.327534,0.184658,1.145924,0.505944,0.907674,0.657046,0.830615,1.112297,...,-0.757614,0.502539,-0.520012,0.301754,0.438154,0.313704,0.310327,0.280511,0.543161,-0.296012
6,-0.082245,0.597984,-0.189503,-0.219884,1.082248,0.693473,-0.236993,1.615453,-0.215960,0.282052,...,0.345772,0.224196,-0.314411,0.302583,0.420569,0.308311,0.305177,0.279298,0.489952,-0.418208
7,-0.030328,0.806234,-0.656965,-0.723606,0.683738,0.887438,-0.809928,0.558230,-0.940033,-0.632425,...,1.298413,-0.006515,0.552262,0.302598,0.375475,0.322828,0.307935,0.268690,0.384329,-0.663870
8,1.894997,0.941752,-1.045917,-1.009627,0.458458,0.940006,1.634661,-0.835563,-1.089341,-1.093812,...,1.596124,-0.962486,1.030882,0.302100,0.378688,0.333434,0.303125,0.263766,0.276433,1.853862
9,1.292405,0.973717,-1.232918,-1.125922,0.321024,0.840103,0.532886,-1.495939,-1.153056,-1.335633,...,-0.048438,-1.225469,0.852808,0.302190,0.325023,0.306391,0.309115,0.319573,0.218486,1.569968


In [54]:
reshaped_train_input = dfToInputRNN(train_input)

In [55]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[-0.67711638,  0.74815575, -1.12518296, -1.11567726,
         -0.20940454,  0.64204408, -1.4657409 , -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
         -1.36143038,  0

In [56]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     112.7290
1     151.5420
2     161.9700
3     261.1200
4     250.5190
5     227.2430
6     198.0696
7     166.8504
8     135.0210
9     123.3470
10    120.1530
11    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [57]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    133.523
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [58]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
12,-2.04338,0.409525,-0.926633,-1.322522,-1.033191,0.589352,-2.665449,1.108405,2.635823,-0.189959,...,-2.260447,-1.348838,1.341235,0.289543,-0.02288,0.173407,0.301984,0.484341,-0.065072,-0.150915


In [59]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[-2.04337971,  0.40952509, -0.9266328 , -1.32252187,
         -1.03319098,  0.58935169, -2.6654494 ,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
         -1.3488375 ,  1

In [60]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [61]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [62]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [63]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[119358834, 3998203414, 1451377463, 2587691631, 4243916338, 2926040709, 1967945710, 1860781810, 3153274977, 2863747618, 3900555741, 256889255, 1572267547, 751030888, 3801655367, 3396002949, 2533049838, 613114628, 1896165667, 1552976691, 3869221598, 1362355860, 2362523177, 3004930957, 3361429443, 896242869, 2972391496, 1521537607, 1112929514, 846084618, 2360124138, 2740956974, 3374142094, 2522102594, 3460757458, 2262528478, 2867056572, 2343414360, 2508561976, 3925212385, 243026508, 4133066106, 3056485179, 385164243, 2230266638, 3632191437, 2234162599, 2684703261, 1423895932, 217604005, 2801862879, 756080348, 1689189492, 3361902179, 4242319469, 3381190450, 1720209487, 2861039027, 3507096834, 3541280234, 3940252583, 2477464710, 4188261264, 3417597948, 1633943997, 2613175106, 1013252655, 3369376960, 657498646, 4160232940, 251759154, 517996385, 3907301469, 3495437397, 2833710462, 23469460, 2009276345, 779767686, 25512539, 3926300428, 1472132766, 3820847147, 904934857, 329810108, 1588173066,

loss: 9.761427879333496


Step: 88 ___________________________________________
loss: 10.33705997467041


Step: 89 ___________________________________________
loss: 5.9788818359375


Step: 90 ___________________________________________
loss: 8.723725318908691


Step: 91 ___________________________________________
loss: 6.627012729644775


Step: 92 ___________________________________________
loss: 8.557332038879395


Step: 93 ___________________________________________
loss: 9.891034126281738


Step: 94 ___________________________________________
loss: 4.944895267486572


Step: 95 ___________________________________________
loss: 6.86298131942749


Step: 96 ___________________________________________
loss: 6.294864654541016


Step: 97 ___________________________________________
loss: 6.325100421905518


Step: 98 ___________________________________________
loss: 9.262235641479492


Step: 99 ___________________________________________
loss: 5.403556823730469


final_seed: 2477464710


In [64]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 31437.5312
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 30095.6973
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 28250.1738
Epoch 4/10000
1/1 [==============================] - 0s 3ms/step - loss: 23080.5098
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 6504.8843
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 3648.8997
Epoch 7/10000
1/1 [==============================] - 0s 3ms/step - loss: 12911.1406
Epoch 8/10000
1/1 [==============================] - 0s 3ms/step - loss: 2043.7396
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 1722.8267
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 2007.0337
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 3892.1379
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - loss: 2644.64

1/1 [==============================] - 0s 4ms/step - loss: 77.3425
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 95.1125
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 71.7181
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 58.5562
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 40.7325
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 119.8294
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 111.7925
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 118.6001
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 27.1683
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 113.2189
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 79.5891
Epoch 110/10000
1/1 [==============================] - 0s 4ms/step - loss: 85.5659
Epoch 111/10000


1/1 [==============================] - 0s 4ms/step - loss: 107.1046
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 37.7461
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 82.1351
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 186.0433
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 90.5392
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 39.3794
Epoch 203/10000
1/1 [==============================] - 0s 5ms/step - loss: 115.2537
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 99.5305
Epoch 205/10000
1/1 [==============================] - 0s 4ms/step - loss: 59.4791
Epoch 206/10000
1/1 [==============================] - 0s 4ms/step - loss: 63.6671
Epoch 207/10000
1/1 [==============================] - 0s 4ms/step - loss: 29.1875
Epoch 208/10000
1/1 [==============================] - 0s 4ms/step - loss: 35.5478
Epoch 209/10000
1

1/1 [==============================] - 0s 5ms/step - loss: 50.2776
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 79.3799
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 42.6062
Epoch 298/10000
1/1 [==============================] - 0s 5ms/step - loss: 27.8018
Epoch 299/10000
1/1 [==============================] - 0s 5ms/step - loss: 49.3684
Epoch 300/10000
1/1 [==============================] - 0s 4ms/step - loss: 48.6782
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 70.2193
Epoch 302/10000
1/1 [==============================] - 0s 5ms/step - loss: 154.4176
Epoch 303/10000
1/1 [==============================] - 0s 4ms/step - loss: 44.6398
Epoch 304/10000
1/1 [==============================] - 0s 4ms/step - loss: 84.8323
Epoch 305/10000
1/1 [==============================] - 0s 5ms/step - loss: 45.4463
Epoch 306/10000
1/1 [==============================] - 0s 5ms/step - loss: 50.0860
Epoch 307/10000
1/1

1/1 [==============================] - 0s 4ms/step - loss: 56.5511
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 74.4502
Epoch 395/10000
1/1 [==============================] - 0s 4ms/step - loss: 40.8450
Epoch 396/10000
1/1 [==============================] - 0s 4ms/step - loss: 37.5851
Epoch 397/10000
1/1 [==============================] - 0s 4ms/step - loss: 114.4296
Epoch 398/10000
1/1 [==============================] - 0s 4ms/step - loss: 38.2039
Epoch 399/10000
1/1 [==============================] - 0s 4ms/step - loss: 99.9118
Epoch 400/10000
1/1 [==============================] - 0s 4ms/step - loss: 73.1058
Epoch 401/10000
1/1 [==============================] - 0s 4ms/step - loss: 97.9677
Epoch 402/10000
1/1 [==============================] - 0s 4ms/step - loss: 101.0825
Epoch 403/10000
1/1 [==============================] - 0s 5ms/step - loss: 101.8110
Epoch 404/10000
1/1 [==============================] - 0s 4ms/step - loss: 92.1509
Epoch 405/10000
1

...

1/1 [==============================] - 0s 3ms/step - loss: 35.3447
Epoch 1467/10000
1/1 [==============================] - 0s 4ms/step - loss: 31.4142
Epoch 1468/10000
1/1 [==============================] - 0s 4ms/step - loss: 31.6577
Epoch 1469/10000
1/1 [==============================] - 0s 4ms/step - loss: 69.0142
Epoch 1470/10000
1/1 [==============================] - 0s 4ms/step - loss: 41.3882
Epoch 1471/10000
1/1 [==============================] - 0s 4ms/step - loss: 38.6371
Epoch 1472/10000
1/1 [==============================] - 0s 4ms/step - loss: 48.0060
Epoch 1473/10000
1/1 [==============================] - 0s 4ms/step - loss: 63.6921
Epoch 1474/10000
1/1 [==============================] - 0s 4ms/step - loss: 56.2312
Epoch 1475/10000
1/1 [==============================] - 0s 4ms/step - loss: 43.0968
Epoch 1476/10000
1/1 [==============================] - 0s 4ms/step - loss: 81.8221
Epoch 1477/10000
1/1 [==============================] - 0s 4ms/step - loss: 32.8414
Epoch 147

1/1 [==============================] - 0s 4ms/step - loss: 13.8875
Epoch 1564/10000
1/1 [==============================] - 0s 4ms/step - loss: 39.6180
Epoch 1565/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.8439
Epoch 1566/10000
1/1 [==============================] - 0s 4ms/step - loss: 52.2409
Epoch 1567/10000
1/1 [==============================] - 0s 4ms/step - loss: 8.4888
Epoch 1568/10000
1/1 [==============================] - 0s 4ms/step - loss: 62.2331
Epoch 1569/10000
1/1 [==============================] - 0s 4ms/step - loss: 46.6479
Epoch 1570/10000
1/1 [==============================] - 0s 4ms/step - loss: 74.2245
Epoch 1571/10000
1/1 [==============================] - 0s 4ms/step - loss: 90.3907
Epoch 1572/10000
1/1 [==============================] - 0s 4ms/step - loss: 74.3763
Epoch 1573/10000
1/1 [==============================] - 0s 4ms/step - loss: 91.2278
Epoch 1574/10000
1/1 [==============================] - 0s 6ms/step - loss: 107.4959
Epoch 157

1/1 [==============================] - 0s 4ms/step - loss: 25.5602
Epoch 1661/10000
1/1 [==============================] - 0s 4ms/step - loss: 54.4660
Epoch 1662/10000
1/1 [==============================] - 0s 4ms/step - loss: 127.6471
Epoch 1663/10000
1/1 [==============================] - 0s 4ms/step - loss: 87.7607
Epoch 1664/10000
1/1 [==============================] - 0s 4ms/step - loss: 95.8845
Epoch 1665/10000
1/1 [==============================] - 0s 4ms/step - loss: 30.1288
Epoch 1666/10000
1/1 [==============================] - 0s 4ms/step - loss: 41.5954
Epoch 1667/10000
1/1 [==============================] - 0s 4ms/step - loss: 55.5418
Epoch 1668/10000
1/1 [==============================] - 0s 4ms/step - loss: 76.1076
Epoch 1669/10000
1/1 [==============================] - 0s 4ms/step - loss: 29.2248
Epoch 1670/10000
1/1 [==============================] - 0s 4ms/step - loss: 42.1506
Epoch 1671/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.2942
Epoch 16

1/1 [==============================] - 0s 4ms/step - loss: 49.6403
Epoch 1758/10000
1/1 [==============================] - 0s 4ms/step - loss: 49.5526
Epoch 1759/10000
1/1 [==============================] - 0s 4ms/step - loss: 43.2128
Epoch 1760/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.8809
Epoch 1761/10000
1/1 [==============================] - 0s 4ms/step - loss: 33.5620
Epoch 1762/10000
1/1 [==============================] - 0s 4ms/step - loss: 22.7411
Epoch 1763/10000
1/1 [==============================] - 0s 4ms/step - loss: 58.2134
Epoch 1764/10000
1/1 [==============================] - 0s 4ms/step - loss: 66.3782
Epoch 1765/10000
1/1 [==============================] - 0s 4ms/step - loss: 50.3848
Epoch 1766/10000
1/1 [==============================] - 0s 4ms/step - loss: 48.0765
Epoch 1767/10000
1/1 [==============================] - 0s 4ms/step - loss: 63.1924
Epoch 1768/10000
1/1 [==============================] - 0s 4ms/step - loss: 82.4459
Epoch 176

1/1 [==============================] - 0s 4ms/step - loss: 28.7953
Epoch 1855/10000
1/1 [==============================] - 0s 4ms/step - loss: 42.3295
Epoch 1856/10000
1/1 [==============================] - 0s 4ms/step - loss: 17.7715
Epoch 1857/10000
1/1 [==============================] - 0s 4ms/step - loss: 20.0588
Epoch 1858/10000
1/1 [==============================] - 0s 4ms/step - loss: 39.9521
Epoch 1859/10000
1/1 [==============================] - 0s 4ms/step - loss: 22.2434
Epoch 1860/10000
1/1 [==============================] - 0s 4ms/step - loss: 34.4389
Epoch 1861/10000
1/1 [==============================] - 0s 4ms/step - loss: 22.9900
Epoch 1862/10000
1/1 [==============================] - 0s 3ms/step - loss: 39.3168
Epoch 1863/10000
1/1 [==============================] - 0s 4ms/step - loss: 43.2056
Epoch 1864/10000
1/1 [==============================] - 0s 4ms/step - loss: 102.6507
Epoch 1865/10000
1/1 [==============================] - 0s 4ms/step - loss: 22.5074
Epoch 18

In [65]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 586ms/step
[157.621](test_target) - [133.35631](prediction) = 24.264692016601572


In [66]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.15394326908598202

In [69]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [70]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,-1.262830,-0.924099,-1.323126,-0.093200,-0.714550,-0.289856,-1.309266,1.148205,-1.122043,-1.121418,...,1.412346,1.235380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.282574
1,0.080102,-0.465059,0.229123,-1.175483,-0.699637,1.343672,0.191634,0.140893,-0.184474,-0.185492,...,-0.643250,-0.021555,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,0.125291
2,1.182728,1.389158,1.094002,1.268682,1.414187,-1.053816,1.117631,-1.289098,1.306518,1.306910,...,-0.769096,-1.213825,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.157284


train_target:


,Amapá - Consumo de Cimento (t)
0,112.729
1,151.542
2,161.970


test_input:


...

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
12,-2.04338,0.409525,-0.926633,-1.322522,-1.033191,0.589352,-2.665449,1.108405,2.635823,-0.189959,...,-2.260447,-1.348838,1.830571,0.75466,-2.18432,-2.362189,1.00512,2.208709,-1.813223,-0.150915


test_target:


,Amapá - Consumo de Cimento (t)
12,157.621


1/1 [==============================] - 1s 559ms/step
Error: 23.086118408203134




[162.80735778808594,
 268.267578125,
 242.8543701171875,
 230.89859008789062,
 198.05714416503906,
 167.33213806152344,
 134.13134765625,
 121.6834716796875,
 120.28743743896484,
 134.53488159179688]

In [71]:
display(targets)
display(predictions)

[261.12,
 250.519,
 227.243,
 198.0696,
 166.85039999999998,
 135.021,
 123.347,
 120.153,
 133.523,
 157.621]

[162.80735778808594,
 268.267578125,
 242.8543701171875,
 230.89859008789062,
 198.05714416503906,
 167.33213806152344,
 134.13134765625,
 121.6834716796875,
 120.28743743896484,
 134.53488159179688]

In [72]:
mae = mean_absolute_error(predictions, targets)
mae

27.66559630737305

In [73]:
porcentage = mae/np.mean(targets)
porcentage

0.1559972432944794